In [226]:
import pandas as pd
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import seaborn as sns
import re
import random
from collections import Counter

import warnings
warnings.filterwarnings('ignore')

---

Третье домашнее задание посвящено достаточно простой, но, надеюсь, интересной задаче, в которой потребуется творчески применить методы сэмплирования

`task 1`

Реализуйте базовый частотный метод по Шерлоку Холмсу:
- подсчитайте частоты букв по корпусам (пунктуацию и капитализацию можно просто опустить, а вот пробелы лучше оставить);
- возьмите какие-нибудь тестовые тексты (нужно взять по меньшей мере 2-3 предложения, иначе вряд ли сработает), зашифруйте их посредством случайной перестановки символов;
- расшифруйте их таким частотным методом.


In [214]:
from string import ascii_lowercase
ascii_lowercase

'abcdefghijklmnopqrstuvwxyz'

In [215]:
alphabet = ' абвгдеёжзийклмнопрстуфхцчшщъыьэюя' # + ascii_lowercase

In [216]:
with open('./corpora/WarAndPeace.txt', 'r') as f:
    text = f.readlines()
    text = str.lower(" ".join(text))
    text = "".join([i for i in text if i in alphabet])
    text = re.sub(r"\W+", " ", text)
    text = re.sub(r"\d+", " ", text)
    text = re.sub(r"ё", "е", text)

In [217]:
text[:1500]

' война и мир самый известный роман льва николаевича толстого как никакое другое произведение писателя отражает глубину его мироощущения и философииэта книга из разряда вечных потому что она обо всем о жизни и смерти о любви и чести о мужестве и героизме о славе и подвиге о войне и мирепервый том знакомит с высшим обществом россии века показаны взаимоотношения между родителями и детьми в семье ростовых сватовство у болконских интриги у безуховых вечера в салоне фрейлины апшерер балы в москве и петербурге лев николаевич толстойчасть первая часть орая часть третья лев николаевич толстой война и мир том часть первая е поместья мой верный раб ну что князь генуа и лукка стали не больше как поместьями фамилии бонапарте нет я вас предупреждаю если вы мне не скажете что у нас война если вы еще позволите себе защищать все гадости все ужасы этого антихриста право я верю что он антихрист я вас больше не знаю вы уж не друг мой вы уж не мой верный раб как вы говорите ну здравствуйте здравствуйте я 

In [218]:
len(text)

642495

In [219]:
def get_counter_unigram(text):
    counter = Counter()
    for symbol in text:
        counter[symbol] += 1
    return counter

In [220]:
wp_counter = get_counter_unigram(text)
print(wp_counter.most_common(5))

[(' ', 103408), ('о', 61282), ('а', 45209), ('е', 42950), ('и', 35838)]


In [221]:
wp_sorted = [i[0] for i in wp_counter.most_common(999)]
print(wp_sorted[:5])

[' ', 'о', 'а', 'е', 'и']


In [222]:
def generate_random_encryption(tokens):
    encrypted = tokens.copy()
    random.shuffle(encrypted)
    encoder = dict(zip(tokens, encrypted))
    return encoder

def encrypt(text, encoder):
    encrypted = str.translate(text, {ord(k): ord(v) for k, v in encoder.items()})
    return encrypted

In [223]:
tokens = list(wp_counter.keys())
encoder = generate_random_encryption(tokens)
print(encoder)

{' ': 'ш', 'в': 'ъ', 'о': 'у', 'й': 'э', 'н': 'и', 'а': 'п', 'и': 'к', 'м': 'ц', 'р': 'о', 'с': 'ф', 'ы': 'н', 'з': 'м', 'е': 'р', 'т': 'т', 'л': 'ж', 'ь': 'е', 'к': 'ю', 'ч': 'й', 'г': 'я', 'д': 'д', 'у': 'ч', 'п': 'а', 'я': 'с', 'ж': 'ь', 'б': 'в', 'щ': 'ы', 'ф': 'щ', 'э': ' ', 'х': 'б', 'ю': 'г', 'ш': 'з', 'ц': 'х', 'ъ': 'л'}


In [196]:
def get_decoder(sample_encoded, get_counter, text):
    
    # считаем частоты в большом тексте
    train_counter = get_counter(text)
    train_keys_sorted = [i[0] for i in train_counter.most_common(999)]

    # считаем частоты в закодированном сообщении
    enc_counter = get_counter(sample_encoded)
    enc_keys_sorted = [i[0] for i in enc_counter.most_common(999)]

    # маппим друг на друга
    decoder = dict(zip(train_keys_sorted, enc_keys_sorted))
    
    return decoder


def decode(sample_encoded, decoder):
    decoded_text = str.translate(sample_encoded, {ord(v): ord(k) for k, v in decoder.items()})
    return decoded_text


def char_accuracy(original_text, decoded_text):
    hit, n = 0, 0
    for orig_char, decoded_char in zip(original_text, decoded_text):
        if orig_char == decoded_char:
            hit += 1
        n += 1
    return hit / n

In [197]:
sample = text[333333:333333 + 700]
print('Исходный текст:\n')
print(sample)

Исходный текст:

началось сражение с фронта но в дюренштейне где я находился войско начало атаку в часу вечера сказал болконский оживляясь и при этом случае предполагая что ему удастся представить уже готовое в его голове правдивое описание всего того что он знал и видел но император улыбнулся и перебил его сколько миль откуда и докуда ваше величество от дюренштейна до кремса три с половиною мили ваше величество французы оставили левый берег как доносили лазутчики в ночь на плотах переправились последние достаточно ли фуража в кремсе фураж не был доставлен в том количестве император перебил его в котором часу убит генерал шмит в семь часов кажется в часов очень печально очень печально император сказал что он


In [198]:
sample_encoded = encrypt(sample, encoder)
print('Закодированное сообщение:\n')
print(sample_encoded)

Закодированное сообщение:

вчхчесбюдбочйрвгрдбдзосвачдвсдъднкорвшартврджнрдщдвчьснгебщдъстбэсдвчхчесдчачэмдъдхчбмдърхрочдбэч чедлсеэсвбэгтдсйгъещщбюдгдяогдфасцдбемхчрдяорнясечжчщдхасдрцмдмнчбабщдяорнбачъгаюдмйрджсасъсрдъдржсджсесърдяочънгъсрдсягбчвгрдъбржсдасжсдхасдсвд вчедгдъгнредвсдгцярочасодмеулвмебщдгдярорлгедржсдбэсеюэсдцгеюдсаэмнчдгднсэмнчдъчшрдърегхрбаъсдсаднкорвшартвчднсдэорцбчдаогдбдясесъгвскдцгегдъчшрдърегхрбаъсдзочвым удсбачъгегдеръутдлрорждэчэднсвсбгегдеч махгэгдъдвсхюдвчдяесачьдяроряочъгегбюдясбернвгрднсбачасхвсдегдзмочйчдъдэорцбрдзмочйдврдлуеднсбачъервдъдасцдэсегхрбаърдгцярочасодярорлгедржсдъдэсасосцдхчбмдмлгаджрврочедшцгадъдбрцюдхчбсъдэчйрабщдъдхчбсъдсхрвюдярхчеювсдсхрвюдярхчеювсдгцярочасодбэч чедхасдсв


In [200]:
decoder = get_decoder(sample_encoded, get_counter_unigram, text)
decoded_text = decode(sample_encoded, decoder)
print('Декодированное сообщение:\n')
print(decoded_text)

Декодированное сообщение:

векенотг требавиа т шровсе во л уюравчсайва ьуа ы вецоуинты лойтдо векено еседп л кетп лакаре тдежен зондовтдий обилныытг и мри эсоя тнпкеа мраумонеьеы ксо аяп пуетсты мраутселисг пба ьосолоа л аьо ьонола мрелуилоа омитевиа лтаьо соьо ксо ов жвен и лиуан во иямаресор пнхзвпнты и маразин аьо тдонгдо яинг осдпуе и уодпуе леча ланикатсло ос уюравчсайве уо драяте сри т моноливою яини леча ланикатсло шревщпжх отселини налхй зараь дед уовотини нежпскиди л вокг ве мносец марамрелинитг мотнаувиа уотсесокво ни шпребе л драята шпреб ва зхн уотселнав л соя доникатсла иямаресор маразин аьо л досороя кетп пзис ьаварен чяис л таяг кетол дебасты л кетол окавг макенгво окавг макенгво иямаресор тдежен ксо ов


In [201]:
print('Точность:\n')
print(f'{char_accuracy(sample, decoded_text):.2%}')

Точность:

36.29%


Прочитать невозможно, но что если взять больше семпл?


In [202]:
for sample_size in [200, 500, 1500, 3000, 5000, 10000, 30000, 50000, 100000, 200000, 400000, 642495]:
    
    # encode
    start_point = 123456
    sample = text[start_point : start_point + sample_size]
    sample_encoded = encrypt(sample, encoder)

    # decode
    decoder = get_decoder(sample_encoded, get_counter_unigram, text)
    decoded_text = decode(sample_encoded, decoder)
    
    # metrics
    char_acc = char_accuracy(sample, decoded_text)
    print(f"sample_size: {sample_size:>7}     accuracy: {char_acc:.0%}")

sample_size:     200     accuracy: 22%
sample_size:     500     accuracy: 38%
sample_size:    1500     accuracy: 45%
sample_size:    3000     accuracy: 51%
sample_size:    5000     accuracy: 64%
sample_size:   10000     accuracy: 66%
sample_size:   30000     accuracy: 82%
sample_size:   50000     accuracy: 69%
sample_size:  100000     accuracy: 70%
sample_size:  200000     accuracy: 91%
sample_size:  400000     accuracy: 92%
sample_size:  642495     accuracy: 92%


Когда размер текста более 5000 символов достигаем какую-никакую точность. На 30000 символах можно практически читать тексты. (Правда все сильно зависит от семла)

`task 2`

Вряд ли в результате получилась такая уж хорошая расшифровка, разве что если вы брали в качестве тестовых данных целые рассказы. Но и Шерлок Холмс был не так уж прост: после буквы E, которая действительно выделяется частотой, дальше он анализировал уже конкретные слова и пытался угадать, какими они могли бы быть. Я не знаю, как запрограммировать такой интуитивный анализ, так что давайте просто сделаем следующий логический шаг:
- подсчитайте частоты биграмм (т.е. пар последовательных букв) по корпусам;
- проведите тестирование аналогично п.1, но при помощи биграмм.


In [224]:
def get_counter_bigram(text):
    counter = Counter()
    for idx, symbol in enumerate(text[:-1]):
        counter[text[idx] + text[idx + 1]] += 1
    return counter

In [204]:
wp_counter_bigram = get_counter_bigram(text)
print(wp_counter_bigram.most_common(15))

[('о ', 13316), ('и ', 11397), ('а ', 10596), ('е ', 10441), (' с', 9863), (' п', 9767), (' в', 9612), (' н', 9347), ('то', 8491), (' о', 7683), ('я ', 7096), (' к', 7045), (' и', 6834), ('ст', 6671), ('ь ', 6603)]


In [205]:
wp_sorted_bigram = [i[0] for i in wp_counter_bigram.most_common(999)]
print(wp_sorted_bigram[:15])

['о ', 'и ', 'а ', 'е ', ' с', ' п', ' в', ' н', 'то', ' о', 'я ', ' к', ' и', 'ст', 'ь ']


In [206]:
def decode_bigram(sample_encoded, decoder):
    """ Декодирование на основе биграмм """
    inversed_decoder = {v: k for k, v in decoder.items()}
    decoded_text = ['.'] * len(sample_encoded)
    for idx, symbol in enumerate(sample_encoded[:-1]):
        bigram = (sample_encoded[idx] + sample_encoded[idx + 1])
        decoded_text[idx] = inversed_decoder[bigram][0]
        decoded_text[idx + 1] = inversed_decoder[bigram][1]
    decoded_text = ''.join(decoded_text)
    return decoded_text

In [207]:
sample = text[333333:333333 + 700]
print('Исходный текст:\n')
print(sample)

Исходный текст:

началось сражение с фронта но в дюренштейне где я находился войско начало атаку в часу вечера сказал болконский оживляясь и при этом случае предполагая что ему удастся представить уже готовое в его голове правдивое описание всего того что он знал и видел но император улыбнулся и перебил его сколько миль откуда и докуда ваше величество от дюренштейна до кремса три с половиною мили ваше величество французы оставили левый берег как доносили лазутчики в ночь на плотах переправились последние достаточно ли фуража в кремсе фураж не был доставлен в том количестве император перебил его в котором часу убит генерал шмит в семь часов кажется в часов очень печально очень печально император сказал что он


In [208]:
sample_encoded = encrypt(sample, encoder)
print('Закодированное сообщение:\n')
print(sample_encoded)

Закодированное сообщение:

вчхчесбюдбочйрвгрдбдзосвачдвсдъднкорвшартврджнрдщдвчьснгебщдъстбэсдвчхчесдчачэмдъдхчбмдърхрочдбэч чедлсеэсвбэгтдсйгъещщбюдгдяогдфасцдбемхчрдяорнясечжчщдхасдрцмдмнчбабщдяорнбачъгаюдмйрджсасъсрдъдржсджсесърдяочънгъсрдсягбчвгрдъбржсдасжсдхасдсвд вчедгдъгнредвсдгцярочасодмеулвмебщдгдярорлгедржсдбэсеюэсдцгеюдсаэмнчдгднсэмнчдъчшрдърегхрбаъсдсаднкорвшартвчднсдэорцбчдаогдбдясесъгвскдцгегдъчшрдърегхрбаъсдзочвым удсбачъгегдеръутдлрорждэчэднсвсбгегдеч махгэгдъдвсхюдвчдяесачьдяроряочъгегбюдясбернвгрднсбачасхвсдегдзмочйчдъдэорцбрдзмочйдврдлуеднсбачъервдъдасцдэсегхрбаърдгцярочасодярорлгедржсдъдэсасосцдхчбмдмлгаджрврочедшцгадъдбрцюдхчбсъдэчйрабщдъдхчбсъдсхрвюдярхчеювсдсхрвюдярхчеювсдгцярочасодбэч чедхасдсв


In [209]:
decoder = get_decoder(sample_encoded, get_counter_bigram, text)
decoded_text = decode_bigram(sample_encoded, decoder)
print('Декодированное сообщение:\n')
print(decoded_text)

Декодированное сообщение:

вю со сяьш  окттаьезсноэонлгои йистквихронатчнаарлвстосаврриндбоеолвю соошуочо и к л  ил    ньоаынснауотеолоиит ксиасиося  елм  и двьдис даелт гдомдоирка о и  к  лпаррелт тпоовеяяккоатме рнааи  пмотмоорлаел озсинаа ыувн ттаиыапмож емока о оирувсн  ивинлнлго еон  у тбккисяукврр  ен т ман пмоьоеоошеосдаоя ело  н  йвго  нипеиаиллрм лпено еайистквихровнйвоестиеннжвм ьеедоорвжгеосдар ипеиаиллрм лпенозс  оруву  поовар чтлбатащ тплеачийвог чар чмыпбб зи и лганялвнемоеосжен тол оварвсяед дт ьттайв поу адгочр зкх  зни естиеаазкх  елнаацынйв пооаткии ж двееорм лпела еон  у тбен т ман пмои еее тндвк л  крмеатекн  снгедеаи ьаижяк лбр еа оьарри к лбр  а кыяен  сомго а кыяен  сомго еон  у тбьоаынснка о от


In [211]:
print('Точность:\n')
print(f'{char_accuracy(sample, decoded_text):.2%}')

Точность:

9.86%


In [212]:
for sample_size in [200, 500, 1500, 3000, 5000, 10000, 30000, 50000, 100000, 200000, 400000, 642495]:
    
    start_point = 123456
    sample = text[start_point:start_point+sample_size]
    
    sample = re.sub(r"[a-z]+", " ", sample)

    sample_encoded = encrypt(sample, encoder)

    decoder = get_decoder(sample_encoded, get_counter_bigram, text)
    decoded_text = decode_bigram(sample_encoded, decoder)

    acc = char_accuracy(sample, decoded_text)
    print(f"sample_size: {sample_size:>7}     accuracy: {acc:.2%}")

sample_size:     200     accuracy: 10.00%
sample_size:     500     accuracy: 10.00%
sample_size:    1500     accuracy: 15.20%
sample_size:    3000     accuracy: 12.80%
sample_size:    5000     accuracy: 10.04%
sample_size:   10000     accuracy: 11.70%
sample_size:   30000     accuracy: 14.60%
sample_size:   50000     accuracy: 15.28%
sample_size:  100000     accuracy: 20.03%
sample_size:  200000     accuracy: 24.88%
sample_size:  400000     accuracy: 38.35%
sample_size:  642495     accuracy: 33.15%


Точность ужасная даже на всем семпле! 

Меня смущает, что в моей реализации для каждого символа (кроме первого и последнего) могут быть два кандидата, от выбора которых очень сильно зависит следующий текст. Я пробовал два простых варианта: пройтись циклом и каждый следующий символ считать началом новой биграммы, и другой подход - нарезать текст на непересекающиеся биграммы. Оба подхода получились одинаково плохими. 

Можно, наверное, как-то обучать биграммы, например, добавить условие, что мы знаем первый или второй символ этой биграммы или вообще посмотреть в сторону скрытых цепей маркова (HMM).

----

`task 3 `

Но и это ещё не всё: биграммы скорее всего тоже далеко не всегда работают. Основная часть задания — в том, как можно их улучшить:

- предложите метод обучения перестановки символов в этом задании, основанный на MCMC-сэмплировании, но по-прежнему работающий на основе статистики биграмм;
- реализуйте и протестируйте его, убедитесь, что результаты улучшились.


Будем ходить в пространстве возможных раскодированных текстов и искать такие, которые более вероятно семплированы из нашего большого корпуса. 

Тогда для каждой перестановки вероятность вытащить ее из нашего текста - то есть ее правдоподобие - это просто перемножение вероятностей каждой биграммы. 

Семплировать будем методом МСМС - двигаемся в каком-то случайном направлении и принимаем или отвергаем гипотезу в зависимости от старого и нового правдоподобия. 

In [30]:
sample = text[333333:333333 + 700]
print('Исходный текст:\n')
print(sample)

Исходный текст:

началось сражение с фронта но в дюренштейне где я находился войско начало атаку в часу вечера сказал болконский оживляясь и при этом случае предполагая что ему удастся представить уже готовое в его голове правдивое описание всего того что он знал и видел но император улыбнулся и перебил его сколько миль откуда и докуда ваше величество от дюренштейна до кремса три с половиною мили ваше величество французы оставили левый берег как доносили лазутчики в ночь на плотах переправились последние достаточно ли фуража в кремсе фураж не был доставлен в том количестве император перебил его в котором часу убит генерал шмит в семь часов кажется в часов очень печально очень печально император сказал что он


In [31]:
sample_encoded = encrypt(sample, encoder)
print('Закодированное сообщение:\n')
print(sample_encoded)

Закодированное сообщение:

ыжфжцмэюьэлжшиы иьэьелмыгжьымькьсалиыйгияыиьпсиьоьыжтмс цэоькмяэбмьыжфжцмьжгжбнькьфжэнькифилжьэбжвжцьумцбмыэб яьмш кцооэюь ьъл ьщгмрьэцнфжиьълисъмцжпжоьфгмьирньнсжэгэоьълисэгжк гюьншиьпмгмкмиькьипмьпмцмкиьължкс кмиьмъ эжы иькэипмьгмпмьфгмьмыьвыжць ьк сицьымь ръилжгмльнцхуынцэоь ьъилиу цьипмьэбмцюбмьр цюьмгбнсжь ьсмбнсжькжйиькиц фиэгкмьмгьсалиыйгияыжьсмьблирэжьгл ьэьъмцмк ымаьр ц ькжйиькиц фиэгкмьелжызнвхьмэгжк ц ьцикхяьуилипьбжбьсмымэ ц ьцжвнгф б ькьымфюьыжьъцмгжтьъилиължк ц эюьъмэцисы иьсмэгжгмфымьц ьенлжшжькьблирэиьенлжшьыиьухцьсмэгжкциыькьгмрьбмц фиэгкиь ръилжгмльъилиу цьипмькьбмгмлмрьфжэньну гьпиыилжцьйр гькьэирюьфжэмкьбжшигэоькьфжэмкьмфиыюьъифжцюымьмфиыюьъифжцюымь ръилжгмльэбжвжцьфгмьмы


In [32]:
CNT_BIGRAM_IN_TEXT = sum(wp_counter_bigram.values())  # 642494

In [33]:
def get_new_mapping(decoder, n_permutations=1):
    new_decoder = decoder.copy()
    for _ in range(n_permutations):
        key_1 = random.choice(list(new_decoder.keys()))
        key_2 = random.choice(list(new_decoder.keys()))
        new_decoder[key_1], new_decoder[key_2] = new_decoder[key_2], new_decoder[key_1]
    return new_decoder

In [34]:
def func_log_likelihood(decoder):
    decoded_text = decode(sample_encoded, decoder)
    log_likelihood = 0
    for idx, symbol in enumerate(decoded_text[:-1]):
        bigram = (decoded_text[idx] + decoded_text[idx + 1])
        bigram_proba = wp_counter_bigram.get(bigram, 1) / CNT_BIGRAM_IN_TEXT
        log_likelihood += np.log(bigram_proba)
    return log_likelihood

In [35]:
def metropolis_hastings_log_accept(cur_l, new_l):
    """ mcmc approach: 
    if p(new) > p(old) then accept with probability = 1
    if p(new) < p(old) then accept with probability = p(new) / p(old)
    """
    if new_l > cur_l:
        return True
    else:
        return np.random.rand() < np.exp(new_l - cur_l)  

In [93]:
def metropolis_hastings(func_log_likelihood, iterations, init_decoder):

    decoded_text = decode(sample_encoded, init_decoder)
    cur_l = best_l = func_log_likelihood(init_decoder)
    cur_decoder = best_decoder = init_decoder

    success_steps = 0
    for i in range(iterations):
        new_decoder = get_new_mapping(cur_decoder)

        new_l = func_log_likelihood(new_decoder)

        if metropolis_hastings_log_accept(cur_l, new_l):
            cur_decoder, cur_l = new_decoder, new_l
            success_steps += 1
            
            if cur_l > best_l:
                best_l = cur_l
                best_decoder = cur_decoder
                
    success_rate = success_steps / iterations

    return best_l, best_decoder, success_rate

In [37]:
init_decoder = get_decoder(sample_encoded, get_counter_unigram, text)

In [38]:
%%time
best_l, best_decoder, sr = metropolis_hastings(func_log_likelihood, iterations=2000, init_decoder=init_decoder)

CPU times: user 2.05 s, sys: 7.94 ms, total: 2.06 s
Wall time: 2.07 s


In [39]:
decoded_text = decode(sample_encoded, best_decoder)

In [40]:
print(f'success steps ratio  {sr:.5f}')
print(f'char accuracy        {char_accuracy(sample, decoded_text):.5f}')
print('\ndecoded text:\n\n', decoded_text)
print()

success steps ratio  0.05850
char accuracy        0.92429

decoded text:

 надалось сражение с шронта но в зюренцтейне гзе я нахозился войско надало атаку в дасу ведера скачал болконский оживляясь и при этом слудае презполагая дто ему узастся презставить уже готовое в его голове правзивое описание всего того дто он чнал и визел но император улыбнулся и перебил его сколько миль откуза и зокуза ваце велидество от зюренцтейна зо кремса три с половиною мили ваце велидество шранщучы оставили левый берег как зоносили лачутдики в нодь на плотах переправились послезние зостатодно ли шуража в кремсе шураж не был зоставлен в том колидестве император перебил его в котором дасу убит генерал цмит в семь дасов кажется в дасов одень педально одень педально император скачал дто он



🔥✅

----

`task 4`

Расшифруйте сообщение:

In [160]:
sample_encoded = '←⇠⇒↟↹↷⇊↹↷↟↤↟↨←↹↝⇛⇯↳⇴⇒⇈↝⇊↾↹↟⇒↟↹⇷⇛⇞↨↟↹↝⇛⇯↳⇴⇒⇈↝⇊↾↹↨←⇌⇠↨↹⇙↹⇸↨⇛↙⇛↹⇠⇛⇛↲⇆←↝↟↞↹⇌⇛↨⇛⇯⇊↾↹⇒←↙⇌⇛↹⇷⇯⇛⇞↟↨⇴↨⇈↹⇠⇌⇛⇯←←↹↷⇠←↙⇛↹↷⇊↹↷⇠←↹⇠↤←⇒⇴⇒↟↹⇷⇯⇴↷↟⇒⇈↝⇛↹↟↹⇷⇛⇒⇙⇞↟↨←↹↳⇴⇌⇠↟↳⇴⇒⇈↝⇊↾↹↲⇴⇒⇒↹⇰⇴↹⇷⇛⇠⇒←↤↝←←↹⇞←↨↷←⇯↨⇛←↹⇰⇴↤⇴↝↟←↹⇌⇙⇯⇠⇴↹↘⇛↨↞↹⇌⇛↝←⇞↝⇛↹↞↹↝↟⇞←↙⇛↹↝←↹⇛↲←⇆⇴⇏'
print(sample_encoded)

←⇠⇒↟↹↷⇊↹↷↟↤↟↨←↹↝⇛⇯↳⇴⇒⇈↝⇊↾↹↟⇒↟↹⇷⇛⇞↨↟↹↝⇛⇯↳⇴⇒⇈↝⇊↾↹↨←⇌⇠↨↹⇙↹⇸↨⇛↙⇛↹⇠⇛⇛↲⇆←↝↟↞↹⇌⇛↨⇛⇯⇊↾↹⇒←↙⇌⇛↹⇷⇯⇛⇞↟↨⇴↨⇈↹⇠⇌⇛⇯←←↹↷⇠←↙⇛↹↷⇊↹↷⇠←↹⇠↤←⇒⇴⇒↟↹⇷⇯⇴↷↟⇒⇈↝⇛↹↟↹⇷⇛⇒⇙⇞↟↨←↹↳⇴⇌⇠↟↳⇴⇒⇈↝⇊↾↹↲⇴⇒⇒↹⇰⇴↹⇷⇛⇠⇒←↤↝←←↹⇞←↨↷←⇯↨⇛←↹⇰⇴↤⇴↝↟←↹⇌⇙⇯⇠⇴↹↘⇛↨↞↹⇌⇛↝←⇞↝⇛↹↞↹↝↟⇞←↙⇛↹↝←↹⇛↲←⇆⇴⇏


In [161]:
arrow_counter = get_counter_unigram(sample_encoded)

In [162]:
arrow_counter.most_common(5)

[('↹', 34), ('←', 22), ('⇛', 22), ('↟', 15), ('⇒', 13)]

In [225]:
len(arrow_counter), len(alphabet)

(28, 34)

In [165]:
init_decoder = get_decoder(sample_encoded, get_counter_unigram, text)

In [166]:
init_decoder

{' ': '↹',
 'о': '←',
 'а': '⇛',
 'е': '↟',
 'и': '⇒',
 'н': '↝',
 'т': '⇴',
 'с': '↨',
 'л': '⇠',
 'в': '⇯',
 'р': '↷',
 'к': '⇌',
 'д': '⇊',
 'м': '⇞',
 'у': '⇈',
 'п': '⇷',
 'я': '↤',
 'г': '↳',
 'ь': '↾',
 'ы': '↙',
 'з': '⇙',
 'б': '↲',
 'ч': '↞',
 'й': '⇆',
 'ж': '⇰',
 'ш': '⇸',
 'х': '↘',
 'ю': '⇏'}

In [173]:
%%time
best_l, best_decoder, sr = metropolis_hastings(func_log_likelihood, iterations=15000, init_decoder=init_decoder)

CPU times: user 4.64 s, sys: 9.55 ms, total: 4.65 s
Wall time: 4.65 s


In [174]:
decoded_text = decode(sample_encoded, best_decoder)

In [175]:
print(f'success steps ratio  {sr:.5f}')
print('\ndecoded text:\n\n', decoded_text[:700])
print()

success steps ratio  0.04800

decoded text:

 если вы вимите норжальный или подти норжальный текст у чтого сообшения который легко продитать скорее всего вы все смелали правильно и полудите жаксижальный балл за послемнее детвертое замание курса хотя конедно я нидего не обешаю



Супер) Даже не пришлось менять код =) 

Дожмем глазами

In [180]:
best_decoder['д'] = '↤'
best_decoder['м'] = '↳'
best_decoder['ж'] = '⇸'
best_decoder['ч'] = '⇞'

In [181]:
decoded_text = decode(sample_encoded, best_decoder)
print('\ndecoded text:\n\n', decoded_text[:700])


decoded text:

 если вы видите нормальный или почти нормальный текст у жтого сообшения который легко прочитать скорее всего вы все сделали правильно и получите максимальный балл за последнее четвертое задание курса хотя конечно я ничего не обешаю
